#  GREENSPOT GROCER DATA
##### python used for refinement of the data 

In [30]:
# Import the pandas library
import pandas as pd

# File path to the input CSV file
file_path = r"C:\Users\manan\Downloads\JTszB4u0TxKOrB0UpjciNw_c12372f001bf4b7fa855abedaebefef1_GreenspotDataset.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Drop the first row of the DataFrame (assuming it's not needed, perhaps a header row)
df = df.drop(0)

# Convert the 'purchase date' column to datetime format, ignoring errors
df['purchase date'] = pd.to_datetime(df['purchase date'], errors='coerce')

# Forward fill missing values in 'purchase date' column
df['purchase date'] = df['purchase date'].ffill()

# Convert the 'date sold' column to datetime format, ignoring errors
df['date sold'] = pd.to_datetime(df['date sold'], errors='coerce')

# Ensure the 'cost' column has consistent values within each 'Item num' group by using the first non-null value
df['cost'] = df.groupby('Item num')['cost'].transform('first')

# Ensure the 'price' column has consistent values within each 'Item num' group by using the first non-null value
df['price'] = df.groupby('Item num')['price'].transform('first')

# Define a function to extract the first word from a string
def extract_first_word(description):
    if isinstance(description, str):  # Check if the input is a string
        words = description.split()   # Split the string into words
        if words:  # If there are words, return the first one
            return words[0]
    return None  # Return None if the input is not a string or if there are no words

# Apply the extract_first_word function to the 'description' column and create a new column 'first word'
df['first word'] = df['description'].apply(extract_first_word)

# Fill in missing 'vendor' values by using the first non-null value within each 'first word' group
df['vendor'] = df.groupby('first word')['vendor'].transform('first')

# Drop the temporary 'first word' column as it's no longer needed
df.drop(columns=['first word'], inplace=True)

# Display the first 30 rows of the DataFrame to check the data
df.head(30)

# Create a DataFrame for vendors with unique values in the 'vendor' column
vendor_df = df[['vendor']].drop_duplicates().reset_index(drop=True)

# Generate a 'vendor id' by taking the first letter of each word in the vendor's name and converting it to uppercase
vendor_df['vendor id'] = vendor_df['vendor'].apply(lambda x: ''.join(word[0] for word in x.split()).upper())

# Reorder the columns and keep only 'vendor id' and 'vendor'
vendor_df = vendor_df[['vendor id', 'vendor']]
vendor_df

# Create a DataFrame for products with unique 'Item num' values and relevant columns
products_df = df[['Item num', 'description', 'item type', 'Unit', 'vendor']].drop_duplicates(subset=['Item num']).reset_index(drop=True)

# Merge the products DataFrame with the vendor DataFrame to associate products with vendor IDs
products_df = products_df.merge(vendor_df, on='vendor', how='left')[['Item num', 'description', 'item type', 'Unit', 'vendor id']]
products_df

# Create a DataFrame for inventory, ensuring unique rows, and add an 'inventory id'
inventory_df = df[['Item num', 'quantity on-hand', 'cost', 'purchase date', 'Location']].drop_duplicates().reset_index(drop=True)
inventory_df['inventory id'] = inventory_df.index + 1  # Create a unique inventory ID by incrementing the index
inventory_df = inventory_df[['inventory id', 'Item num', 'quantity on-hand', 'cost', 'purchase date', 'Location']]
inventory_df

# Create a DataFrame for sales, dropping rows with missing 'date sold' values
sales_df = df[['Item num', 'date sold', 'cust', 'Quantity', 'price']].dropna(subset=['date sold']).reset_index(drop=True)

# Generate a unique 'sale id' by combining 'Item num' and 'date sold', removing unwanted characters
sales_df['sale id'] = sales_df['Item num'].astype(str).str.replace('.0', '') + sales_df['date sold'].astype(str).str.replace('-', '')
sales_df = sales_df[['sale id', 'Item num', 'date sold', 'cust', 'Quantity', 'price']]
sales_df

# Save the modified DataFrame to a new CSV file
df.to_csv(r"C:\Users\manan\Desktop\GREENSPOT GROCER.csv", index=False)

# Save the vendors DataFrame to a CSV file
vendor_df.to_csv(r"C:\Users\manan\Desktop\vendor.csv", index=False)

# Save the products DataFrame to a CSV file
products_df.to_csv(r"C:\Users\manan\Desktop\products.csv", index=False)

# Save the inventory DataFrame to a CSV file
inventory_df.to_csv(r"C:\Users\manan\Desktop\inventory.csv", index=False)

# Save the sales DataFrame to a CSV file
sales_df.to_csv(r"C:\Users\manan\Desktop\sales.csv", index=False)
